# Photometry in pycoco

---------------------

## Introduction
This notebook aims to serve as a short tutorial (along with the others in this directory). In this notebook, the tools for adding/testing/plotting photometry in **`pycoco`**, as a companion to the **`c`** code **`CoCo`**.

**`CoCo`** takes photometry data in one input phot file. **`pycoco`** can be used to (*hopefully*) easily wrangle your exisiting photometry into the correct format.

In [1]:
from __future__ import print_function ## Force python3-like printing

%matplotlib inline
from matplotlib import pyplot as plt


try:
    from importlib import reload
except:
    pass

import os
import numpy as np
from astropy.table import Table

import pycoco as pcc

reload(pcc) ## FOR DEV

<module 'pycoco' from '/Users/berto/Code/verbose-enigma/pycoco/__init__.py'>

## Using `PhotometryClass`
----

### `PhotometryClass` methods and variables

Photometry in **`pycoco`** is handled using **`PhotometryClass`** which has the methods and variables:

**`PhotometryClass.__init__`**

**`PhotometryClass._get_data_directory`**

**`PhotometryClass._combine_phot`**


**`PhotometryClass.set_data_directory`**

**`PhotometryClass.load_phot_from_file`**

**`PhotometryClass.load_phot_ap_tables`**

**`PhotometryClass.load`**

**`PhotometryClass.save`**

**`PhotometryClass.plot`**

**`PhotometryClass.save_phot`**


**`PhotometryClass._default_data_dir_path`**

----
# I/O

## File Input
----
### Input from Files for each Filter

You can load in phot files one by one, or you can use the **`PhotometryClass.load_phot_from_files()`** method, to find all of those that match a SN, or you can load in an appropriately formatted file with **`PhotometryClass.load_phot()`**. We will try first the individual phot files formatted `$SNNAME_$FILTERNAME.dat`.

In [2]:
P = pcc.PhotometryClass()

P.load_phot_from_files()

/Users/berto/Code/verbose-enigma/pycoco/classes.py:958: UserWarning: Provide a SN name
  warnings.warn("Provide a SN name")


Oops! We need to provide a SN name!

In [3]:
P.load_phot_from_files(snname = 'SN2005bf')

NameError: name 'find_filter_phot' is not defined

The operation has run with `verbose = True`, so it has spat out quite a bit of information. Namely, it found 6 files in the (default) data directory. 
5 of these matched the specified SN name (SN2005bf). The full filepaths of these was printed as the files were iterated over and loaded in to **`astropy`** tables. The filters from the file data and file names are printed, and all seem to match.

if a warning raised, it is because a file matching the naming convention ($SNNAME.dat) of correctly formatted datafile was found, and that might be easier to deal with!

All of the individual filter files are read into tables, and unpacked into a combined phot instance. The equivalent operation for a photfile containing all of the photometry is:

In [ ]:
P = pcc.PhotometryClass()

phot_path = "/Users/berto/Code/verbose-enigma/testdata/lc/SN2005bf.dat"
P.load(phot_path)

Let's take a look at what is now in our **`PhotometryClass`** instance:

In [ ]:
P.__dict__.keys()

Let's look at the data that was loaded in:

In [ ]:
P.data.keys()

In [ ]:
P.data['BessellB'].show_in_notebook(display_length = 10)

This is good - all of the found photometry has been read in. 

## Input From Astropy Tables
----
If you have photometry that is already in **`astropy`** tables, then you can load it in using **`PhotometryClass.load_phot_ap_tables()`**. There must be columns labelled `MJD`, `flux`, `flux_err` and `filter`.

## Input From Single **`CoCo`**-format file
----
**`CoCo`** takes as an input, one photometry file, with all of the data inside it. So far, we have used **`pycoco`** to read in several *different* photometry files. 

The **`CoCo`** standard format is the same columns as the individual ones (`MJD`, `flux`, `flux_err` and `filter`), but all of the photometry is in together. When using  **`PhotometryClass.load_phot_from_files()`**, an **`astropy`** table matching this is generated, and stored in **`PhotometryClass.phot`**. 

In [ ]:
P.phot.show_in_notebook(display_length = 10)

It is ordered by time (MJD).

If the photometry is already in this format, and you want to interact with it, you can use **`pycoco.load()`**, which is a wrapper for **`pycoco.load_formatted_phot()`**.

In [ ]:
P_new = pcc.load("/Users/berto/Code/verbose-enigma/testdata/lc/SN2005bf.dat", verbose = False)
P_new.phot.show_in_notebook(display_length = 10)

This uses **`PhotometryClass.unpack()`** to convert back from this format to individual filter files, putting them in an `OrderedDict`, **`PhotometryClass.data`**.

In [ ]:
P_new.unpack()

In [ ]:
P_new.data.keys()

In [ ]:
P_new.data_filters

# File Output
----

File out to a directory of choice can be done using **`PhotometryClass.save()`**, using the default directory: 

In [ ]:
P.save("SN2005bf.dat", squash = True)

# Plotting Photometry
----

Once the photometry is loaded in, you can view it using **`PhotometryClass.plot()`**, which accepts **`matplotlib.pyplot`** **`args`** and **`kwargs`**.

In [ ]:
P.plot()

Looks good!

## Associating **`FilterClass`** with Photometry
----

What we will demonstrate now is associating **`FilterClass`** with the photometry. **`PhotometryClass`** knows about the default filter directory (but another can be specified using **`PhotometryClass.set_filter_directory`**):

In [ ]:
P.filter_directory

The **`PhotometryClass.load_phot_from_files()`** and **`PhotometryClass.load_formatted_phot()`** (the latter is used by **`pcc.load()`**) methods automatically search the filter directory for the appropriate filters using **`pcc.load_filter()`**, and loads them in. For more on **`pycocoo.FilterClass`** see `pycoco_adding_filters.ipynb`.

If we take a look, we can see that we have 5 **`FilterClass`** instances, one for each of the photometry files.

In [ ]:
P.data_filters

We can have a look at the spectral coverage of the photometry by using the **`PhotometryClass.plot_filters()`** method:

In [ ]:
P.plot_filters(legend = True)

This looks good! We have good spectral coverage.